# Data Loading

In [194]:
import torch
import torch.nn as nn
from data.dataset import data_loader

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [195]:
train_loader, val_loader = data_loader(
    data_dir='./data/datasets', batch_size=64
)
test_loader = data_loader(
    data_dir='./data/datasets', batch_size=64,
    test=True
)

len(train_loader), len(val_loader), len(test_loader)

Files already downloaded and verified
Files already downloaded and verified


(704, 79, 157)